In [ ]:
import os
os.chdir("/content/drive/MyDrive/Colab Notebooks/Python_Scripts/Assignments/vertisystem/")
print(os.getcwd())

/content/drive/MyDrive/Colab Notebooks/Python_Scripts/Assignments/vertisystem


In [ ]:
import json
import random
import os
import datetime
from concurrent.futures import ThreadPoolExecutor
import time

# Constants
N = 5000  # Number of JSON files to generate
M_MIN, M_MAX = 50, 100  # Range for number of flight records per file
K = 150  # Number of cities
L = 0.007  # Probability for NULL values
OUTPUT_DIR = os.getcwd()+"/tmp/flights"

In [ ]:
OUTPUT_DIR

'/content/drive/MyDrive/Colab Notebooks/Python_Scripts/Assignments/vertisystem/tmp/flights'

In [ ]:


# Generate a list of city names
cities = [f"City_{i}" for i in range(K)]

def generate_flight_record():
    origin, destination = random.sample(cities, 2)
    date = datetime.date(2024, random.randint(1, 12), random.randint(1, 28))
    duration = random.randint(1800, 36000)  # 30 minutes to 10 hours
    passengers = random.randint(50, 300)

    record = {
        "date": date.isoformat(),
        "origin_city": origin,
        "destination_city": destination,
        "flight_duration_secs": duration,
        "passengers_on_board": passengers
    }

    # Introduce NULL values with probability L
    if random.random() < L:
        key = random.choice(list(record.keys()))
        record[key] = None

    return record

def generate_json_file(file_number):
    month = random.randint(1, 12)
    year = 24
    origin_city = random.choice(cities)
    filename = f"{OUTPUT_DIR}/{month:02d}-{year:02d}-{origin_city}-flights-{file_number}.json"
    os.makedirs(os.path.dirname(filename), exist_ok=True)

    records = [generate_flight_record() for _ in range(random.randint(M_MIN, M_MAX))]

    with open(filename, 'w') as f:
        json.dump(records, f)

def analyze_files():
    total_records = 0
    dirty_records = 0
    flight_durations = {}
    city_passengers = {city: {"arrived": 0, "left": 0} for city in cities}

    for filename in os.listdir(OUTPUT_DIR):
        with open(os.path.join(OUTPUT_DIR, filename), 'r') as f:
            records = json.load(f)
            total_records += len(records)

            for record in records:
                if None in record.values():
                    dirty_records += 1
                else:
                    dest = record["destination_city"]
                    if dest not in flight_durations:
                        flight_durations[dest] = []
                    flight_durations[dest].append(record["flight_duration_secs"])

                    city_passengers[record["origin_city"]]["left"] += record["passengers_on_board"]
                    city_passengers[record["destination_city"]]["arrived"] += record["passengers_on_board"]

    # Calculate AVG and P95 for top 25 destination cities
    top_25_destinations = sorted(flight_durations.items(), key=lambda x: len(x[1]), reverse=True)[:25]
    stats = {}
    for dest, durations in top_25_destinations:
        avg = sum(durations) / len(durations)
        p95 = sorted(durations)[int(len(durations) * 0.95)]
        stats[dest] = {"avg": avg, "p95": p95}

    # Find cities with max passengers arrived and left
    max_arrived = max(city_passengers.items(), key=lambda x: x[1]["arrived"])
    max_left = max(city_passengers.items(), key=lambda x: x[1]["left"])

    return total_records, dirty_records, stats, max_arrived, max_left

def main():
    start_time = time.time()

    # Generation phase
    print(f"Generating {N} JSON files...")
    with ThreadPoolExecutor() as executor:
        executor.map(generate_json_file, range(N))

    # Analysis & Cleaning phase
    print("Analyzing files...")
    total_records, dirty_records, flight_stats, max_arrived, max_left = analyze_files()

    end_time = time.time()
    total_duration = end_time - start_time

    # Print results
    print(f"Total records processed: {total_records}")
    print(f"Dirty records: {dirty_records}")
    print(f"Total run duration: {total_duration:.2f} seconds")

    print("\nAVG and P95 of flight duration for Top 25 destination cities:")
    for dest, stats in flight_stats.items():
        print(f"{dest}: AVG = {stats['avg']:.2f} seconds, P95 = {stats['p95']:.2f} seconds")

    print(f"\nCity with max passengers arrived: {max_arrived[0]} ({max_arrived[1]['arrived']} passengers)")
    print(f"City with max passengers left: {max_left[0]} ({max_left[1]['left']} passengers)")

if __name__ == "__main__":
    main()

Generating 5000 JSON files...
Analyzing files...
Total records processed: 374941
Dirty records: 2708
Total run duration: 204.92 seconds

AVG and P95 of flight duration for Top 25 destination cities:
City_76: AVG = 18962.84 seconds, P95 = 34263.00 seconds
City_132: AVG = 18780.34 seconds, P95 = 33953.00 seconds
City_12: AVG = 18900.60 seconds, P95 = 34435.00 seconds
City_133: AVG = 18701.48 seconds, P95 = 34168.00 seconds
City_113: AVG = 19245.64 seconds, P95 = 34351.00 seconds
City_121: AVG = 18681.55 seconds, P95 = 34117.00 seconds
City_126: AVG = 18880.62 seconds, P95 = 34163.00 seconds
City_102: AVG = 19082.48 seconds, P95 = 34234.00 seconds
City_77: AVG = 18635.49 seconds, P95 = 34315.00 seconds
City_75: AVG = 18805.13 seconds, P95 = 34251.00 seconds
City_68: AVG = 18837.14 seconds, P95 = 34250.00 seconds
City_6: AVG = 18847.60 seconds, P95 = 34324.00 seconds
City_144: AVG = 18965.15 seconds, P95 = 34204.00 seconds
City_112: AVG = 19086.26 seconds, P95 = 34284.00 seconds
City_91: A